In [10]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import PIL 
from PIL import Image
from keras_preprocessing import image

from numpy import asarray

from tensorflow.keras import Input, layers

print(tf.__version__)


2.4.1


In [6]:
dataset = "cifar"
if dataset == "fashion_mnist":
  fashion_mnist = tf.keras.datasets.fashion_mnist
  (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
  class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
elif dataset == "mnist":
  mnist = tf.keras.datasets.mnist
  (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
  class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
elif dataset == "cifar":
  (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
  class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
  train_labels = train_labels.squeeze()
  test_labels = test_labels.squeeze()

input_shape = train_images.shape[1:]
train_images = train_images / 255.0
test_images = test_images / 255.0
print("Training Images range from {:2.5f} to {:2.5f}".format(np.min(train_images), np.max(train_images)))
print("Test     Images range from {:2.5f} to {:2.5f}".format(np.min(test_images), np.max(test_images)))

Training Images range from 0.00000 to 1.00000
Test     Images range from 0.00000 to 1.00000


In [3]:
print (input_shape)

(32, 32, 3)


In [4]:
model = tf.keras.Sequential([
    Input(shape=input_shape),
    layers.Conv2D(32, kernel_size=(3,3), activation="relu", padding='same'),
    layers.Conv2D(64, kernel_size=(3,3), activation="relu", padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),    
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(10)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              16778240  
_________________________________________________________________
dense_1 (Dense)              (None, 10)                10250     
Total params: 16,807,882
Trainable params: 16,807,882
Non-trainable params: 0
____________________________________________

In [7]:
train_hist = model.fit(train_images, train_labels, epochs=50)
model.save('saved_models/cifar_cnn_model')

Epoch 1/50
1563/1563 [==============================] - 191s 122ms/step - loss: 1.5141 - accuracy: 0.4572
Epoch 2/50
1563/1563 [==============================] - 184s 118ms/step - loss: 0.8554 - accuracy: 0.7009
Epoch 3/50
1563/1563 [==============================] - 209s 134ms/step - loss: 0.5438 - accuracy: 0.8122
Epoch 4/50
1563/1563 [==============================] - 185s 118ms/step - loss: 0.2628 - accuracy: 0.9112
Epoch 5/50
1563/1563 [==============================] - 186s 119ms/step - loss: 0.1074 - accuracy: 0.9656
Epoch 6/50
1563/1563 [==============================] - 184s 118ms/step - loss: 0.0678 - accuracy: 0.9792
Epoch 7/50
1563/1563 [==============================] - 185s 118ms/step - loss: 0.0509 - accuracy: 0.9843
Epoch 8/50
1563/1563 [==============================] - 184s 118ms/step - loss: 0.0502 - accuracy: 0.9842
Epoch 9/50
1563/1563 [==============================] - 181s 116ms/step - loss: 0.0446 - accuracy: 0.9857
Epoch 10/50
1563/1563 [=======================

In [8]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

313/313 - 5s - loss: 5.7021 - accuracy: 0.6645

Test accuracy: 0.6644999980926514


In [30]:
#resizing to 32x32
im = Image.open('/Users/Amanda/Desktop/airplane.jpg')
newsize = (32, 32) 
resized_im = im.resize(newsize)   
resized_im.show()  

In [35]:
#processing
im = (np.expand_dims(resized_im,0)) #batch
prediction = model.predict(resized_im) #process

print(prediction)

np.argmax(prediction[0]) #"grab" prediction out of list


ValueError: Failed to find data adapter that can handle input: <class 'PIL.Image.Image'>, <class 'NoneType'>

In [33]:
model_2 = tf.keras.Sequential([
    Input(shape=input_shape),
    layers.DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding='same'),
    layers.Conv2D(32, kernel_size=(1,1), strides=(1,1), activation="relu", padding='same'),
    layers.DepthwiseConv2D(kernel_size=(3,3), strides=(2,2), padding='same'),
    layers.Conv2D(64, kernel_size=(1,1), strides=(1,1), activation="relu", padding='same'),
    layers.MaxPooling2D(pool_size=(2, 2)),    
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dense(10)
]) 

model_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
depthwise_conv2d (DepthwiseC (None, 16, 16, 3)         30        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        128       
_________________________________________________________________
depthwise_conv2d_1 (Depthwis (None, 8, 8, 32)          320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          2112      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)             